In [1]:
import numpy as np 
import pandas as pd
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
!conda install -c anaconda lxml --yes
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 anaconda --> 2019.11.28-py36_0 conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1-h7b6447c_0  anaconda --> 1.1.1d-h516909a_0 conda-forge


certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newe

In [6]:
#assign a name to the wikipedia link
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#read the table from the link
torontotable = pd.read_html(link)[0]
#get the indexes where the borough is not assigned to a postcode
indexBorough = torontotable[ torontotable['Borough'] == "Not assigned" ].index
#delete rows where borough is not assigned
torontotable.drop(indexBorough , inplace=True)
#get the indexes where the neighbourhood is not assigned to a postcode
indexNeigh = torontotable[ torontotable['Neighbourhood'] == "Not assigned" ].index

#change the not assigned neighbourhoods with the borough name
for i in indexNeigh:
    torontotable.at[i, 'Neighbourhood'] = torontotable.at[i, 'Borough']
    
#group by the borough and join the neighbourhood with commas    
TorontoLast=torontotable.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])

#reset the index to start from 0
TorontoLast=TorontoLast.reset_index()


#read the given csv file
longlat=pd.read_csv("http://cocl.us/Geospatial_data")

#change the column name to make both dataframe same column name Postcode
longlat.rename(columns={'Postal Code':'Postcode'},inplace=True)

#merge the two dataframe to get the longitude and latitude values
TorontoLast=TorontoLast.merge(longlat)
TorontoLast

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [3]:
CLIENT_ID = 'OZRADFIBAZGBR2245GCYZMV5T3CVT1S3MGXAA2MSMVLH0RM2' # Foursquare ID
CLIENT_SECRET = '0Y5UJELPFTJMYFWP2CRN0HEF52PRHTVA4PXHUVC4TVEUZNQN' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OZRADFIBAZGBR2245GCYZMV5T3CVT1S3MGXAA2MSMVLH0RM2
CLIENT_SECRET:0Y5UJELPFTJMYFWP2CRN0HEF52PRHTVA4PXHUVC4TVEUZNQN


In [7]:
etobicokedata = TorontoLast[TorontoLast['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicokedata.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999


In [12]:
#get the latitude and longitude of chosen borough etobicoke.
latitude = etobicokedata.at[1,'Latitude']
longitude = etobicokedata.at[1,'Longitude']
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.60241370000001, -79.54348409999999.


In [22]:
# create map of Etobicoke using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicokedata['Latitude'], etobicokedata['Longitude'], etobicokedata['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

In [15]:
#Now, let's get the top 100 venues that are in Etobicoke within a radius of 5km
LIMIT = 100
radius = 5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=OZRADFIBAZGBR2245GCYZMV5T3CVT1S3MGXAA2MSMVLH0RM2&client_secret=0Y5UJELPFTJMYFWP2CRN0HEF52PRHTVA4PXHUVC4TVEUZNQN&v=20180605&ll=43.60241370000001,-79.54348409999999&radius=5000&limit=100'

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    
results = requests.get(url).json()

In [20]:
#Now we are ready to clean the json and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Farm Boy,Grocery Store,43.610012,-79.547581
1,Il Paesano Pizzeria & Restaurant,Pizza Place,43.601280,-79.545028
2,Woody's Burgers bar and grill,Burger Joint,43.592424,-79.541825
3,Indigo,Bookstore,43.613365,-79.557980
4,Fair Grounds Cafe & Roastery,Café,43.592465,-79.541579


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.
